In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
raw_data = pd.read_csv(r'C:\Users\shubh\Downloads\uspollution\pollution_us_2000_2016.csv')

In [3]:
raw_data_copy = raw_data.copy() # just so I don't mess up the real data

In [4]:
(raw_data_copy.isna().sum()/len(raw_data_copy)*100) # %age of NA values in each column

Unnamed: 0            0.000000
State Code            0.000000
County Code           0.000000
Site Num              0.000000
Address               0.000000
State                 0.000000
County                0.000000
City                  0.000000
Date Local            0.000000
NO2 Units             0.000000
NO2 Mean              0.000000
NO2 1st Max Value     0.000000
NO2 1st Max Hour      0.000000
NO2 AQI               0.000000
O3 Units              0.000000
O3 Mean               0.000000
O3 1st Max Value      0.000000
O3 1st Max Hour       0.000000
O3 AQI                0.000000
SO2 Units             0.000000
SO2 Mean              0.000000
SO2 1st Max Value     0.000000
SO2 1st Max Hour      0.000000
SO2 AQI              49.975754
CO Units              0.000000
CO Mean               0.000000
CO 1st Max Value      0.000000
CO 1st Max Hour       0.000000
CO AQI               49.999571
dtype: float64

In [6]:
columns_to_observe = list(range(1, 9))+[23, 28]
columns_to_observe

[1, 2, 3, 4, 5, 6, 7, 8, 23, 28]

In [7]:
raw_data_copy.iloc[:, columns_to_observe].head() 

,State Code,County Code,Site Num,Address,State,County,City,Date Local,SO2 AQI,CO AQI
0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,13.0,NaN
1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,13.0,25.0
2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,NaN,NaN
3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,NaN,25.0
4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,4.0,NaN


In [13]:
raw_data_copy.Address.value_counts().head() # number of instances of each address

PIKE AVE AT RIVER ROAD          35332
1130 EASTWAY DRIVE              32012
5888 MISSION BLVD., RUBIDOUX    30178
1630 N MAIN ST, LOS ANGELES     25225
2956-A TREAT BOULEVARD          23686
Name: Address, dtype: int64

In [9]:
raw_data_copy[raw_data_copy['Site Num'] == 3002].Address.value_counts() #Same site num has 2 address values

1645 E ROOSEVELT ST-CENTRAL PHOENIX STN    22568
1167 NORTH DOVER ROAD                       2572
Name: Address, dtype: int64

In [11]:
raw_data_copy['State County Site Code'] = raw_data_copy['State Code'].astype(str) + '-' + raw_data_copy['County Code'].astype(str) + '-' + raw_data_copy['Site Num'].astype(str)
raw_data_copy.head() # Added a new column with state, county and site code combined

,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,State County Site Code
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,3.000000,9.0,21,13.0,Parts per million,1.145833,4.2,21,NaN,4-13-3002
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,3.000000,9.0,21,13.0,Parts per million,0.878947,2.2,23,25.0,4-13-3002
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,2.975000,6.6,23,NaN,Parts per million,1.145833,4.2,21,NaN,4-13-3002
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,2.975000,6.6,23,NaN,Parts per million,0.878947,2.2,23,25.0,4-13-3002
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,1.958333,3.0,22,4.0,Parts per million,0.850000,1.6,23,NaN,4-13-3002


In [14]:
raw_data_copy['State County Site Code'].value_counts().head()

5-119-7      35332
37-119-41    32012
6-65-8001    30178
6-37-1103    25225
6-13-2       23686
Name: State County Site Code, dtype: int64